In [1]:
from Mesh import Mesh
import torch.nn.functional as tnn
import torch
use_cuda = 0
torchdeviceId = torch.device('cuda:0') if use_cuda else 'cpu'
torchdtype = torch.float64
import time 
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
M=Mesh(None,None,"data/icos.ply")

In [3]:

mu1=torch.exp(-5*((M.V - torch.as_tensor([0.0,1.0,0.0]))**2).sum(dim=1))
mu1=M.normalize(mu1)
mu2=torch.exp(-5*((M.V - torch.as_tensor([0.0,0.0,-1.0]))**2).sum(dim=1))
mu2=M.normalize(mu2)
M.plot_pair(mu1,mu2)

(12, 4)


In [4]:
T=12
match_coeff=1e6
path_coeff=1


In [5]:
def L(out):
    out=torch.unsqueeze(out,0)
    weight = torch.Tensor([[[[10,10,10],[10,0,10],[10,10,10]]]]).to(dtype=torchdtype, device=torchdeviceId)
    out = tnn.conv2d(out,weight.repeat(2,1,1,1), padding='same',groups=2)
    #out = tnn.conv2d(out,weight.repeat(2,1,1,1), padding='same',groups=2)
    #out = tnn.conv2d(out,weight.repeat(2,1,1,1), padding='same',groups=2)
    return out[0]

In [6]:
start =time.time()
vecs,ps = M.OT(mu1.flatten(),mu2.flatten(),T,match_coeff,path_coeff,max_iter=1000)
pl_pathlength = M.path_length(mu1.flatten(),vecs)
N=vecs.shape[0]

In [7]:
M.plot_sequence(ps,vecs)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [8]:
print(pl_pathlength.sum())
print(pl_pathlength)

tensor(0.2620, dtype=torch.float64)
tensor([0.0249, 0.0216, 0.0195, 0.0190, 0.0207, 0.0230, 0.0247, 0.0258, 0.0265,
        0.0277, 0.0286], dtype=torch.float64)
